<a href="https://colab.research.google.com/github/jaemmy425/Jaemmy425.github.io/blob/main/%E5%89%91%E6%A1%A5%E5%9C%B0%E7%90%86%E7%A9%BA%E9%97%B4%E5%88%86%E6%9E%90%E5%89%8D%E5%9B%9B%E6%AD%A5%E4%BB%A3%E7%A0%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 单元格 1: 安装所有必要的库
!pip install geopandas fiona shapely pyproj rtree

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 28.5 MB/s eta 0:00:00


In [5]:
# 单元格 2: 解压您的数据文件
!unzip /content/Cambridgecity_clip -d /content/
!unzip /content/Cambridge_OS_Buildings -d /content/ # 如果您有另一个OS建筑数据的zip文件，请确保文件名正确

Archive:  /content/Cambridgecity_clip.zip
  inflating: /content/Cambridgecity_clip.prj  
  inflating: /content/Cambridgecity_clip.shp  
  inflating: /content/Cambridgecity_clip.shx  
  inflating: /content/Cambridgecity_clip.cpg  
  inflating: /content/Cambridgecity_clip.dbf  
Archive:  /content/Cambridge_OS_Buildings.zip
replace /content/Cambridge_OS_Buildings.cpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/Cambridge_OS_Buildings.cpg  
  inflating: /content/Cambridge_OS_Buildings.dbf  
  inflating: /content/Cambridge_OS_Buildings.prj  
  inflating: /content/Cambridge_OS_Buildings.qmd  
  inflating: /content/Cambridge_OS_Buildings.shp  
  inflating: /content/Cambridge_OS_Buildings.shx  


In [6]:
# 单元格 3: 您的核心地理空间分析代码

import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import transform
from functools import partial
import pyproj # 修正：from pypro 改为 import pyproj

try:
    # --- 1. 数据加载 ---
    # 替换为您的实际文件路径
    osm_buildings_path = "/content/Cambridgecity_clip.shp"
    os_buildings_path = "/content/Cambridge_OS_Buildings.shp"

    print("正在尝试读取数据...")
    df_osm = gpd.read_file(osm_buildings_path)
    df_os = gpd.read_file(os_buildings_path)
    print("数据读取成功。")

    # --- 数据加载后的初步检查 ---
    print("\n--- 数据加载检查 ---")
    if df_osm.empty:
        print(f"警告：OSM 数据集 '{osm_buildings_path}' 为空，可能未读取成功或文件中没有要素。")
        raise ValueError("OSM 数据加载失败或为空。请检查文件路径和内容。")
    else:
        print(f"成功读取 OSM 数据：包含 {len(df_osm)} 个要素。")
        print("OSM 数据集预览 (前5行):")
        print(df_osm.head())
        print(f"OSM 数据集 CRS: {df_osm.crs}")
        print(f"OSM 数据集几何类型: {df_osm.geometry.geom_type.unique()}")


    if df_os.empty:
        print(f"警告：OS 数据集 '{os_buildings_path}' 为空，可能未读取成功或文件中没有要素。")
        raise ValueError("OS 数据加载失败或为空。请检查文件路径和内容。")
    else:
        print(f"成功读取 OS 数据：包含 {len(df_os)} 个要素。")
        print("OS 数据集预览 (前5行):")
        print(df_os.head())
        print(f"OS 数据集 CRS: {df_os.crs}")
        print(f"OS 数据集几何类型: {df_os.geometry.geom_type.unique()}")
        print("df_os 的几何列名:", df_os.geometry.name)


    # --- 详细检查 df_osm 的几何列 ---
    print("\n--- 详细检查 df_osm 的几何列 ---")
    print("df_osm 类型:", type(df_osm))
    print("df_osm 是否为 GeoDataFrame:", isinstance(df_osm, gpd.GeoDataFrame))
    if not df_osm.empty:
        print("df_osm 列名:", df_osm.columns.tolist())
        try:
            # 确保几何列是活跃的
            if df_osm.geometry.name != 'geometry':
                print(f"警告：df_osm 的默认几何列不是 'geometry'，而是 '{df_osm.geometry.name}'。将其设置为 'geometry'。")
                df_osm = df_osm.set_geometry(df_osm.geometry.name)

            print("df_osm 几何列名称:", df_osm.geometry.name)
            print("df_osm 几何列数据类型:", df_osm.geometry.dtype)
            print("df_osm 几何列前5行:")
            print(df_osm.geometry.head())
            print("df_osm 几何类型唯一值:", df_osm.geometry.geom_type.unique())
            num_invalid_geom_osm = (~df_osm.geometry.is_valid).sum()
            print(f"df_osm 中无效几何体数量: {num_invalid_geom_osm}")
            if num_invalid_geom_osm > 0:
                print("警告：df_osm 包含无效几何体，将尝试修复。")
                df_osm['geometry'] = df_osm['geometry'].buffer(0) # 修复无效几何
                print("df_osm 无效几何体已尝试修复。")

        except AttributeError:
            print("错误：df_osm 似乎没有默认的 'geometry' 列。")
            print("请检查原始 OSM Shapefile 是否包含几何信息，以及 GeoPandas 是否正确识别。")
            raise ValueError("df_osm 中缺少几何列。请检查OSM数据。")
    print("--- df_osm 几何列检查结束 ---")


    # --- 2. 投影检查与统一 ---
    print("\n--- 投影统一检查与执行 ---")
    if df_osm.crs != df_os.crs:
        print(f"OSM CRS: {df_osm.crs}, OS CRS: {df_os.crs}")
        print("警告：两个数据集的 CRS 不匹配，将尝试将 OSM 数据投影到 OS 数据的 CRS。")
        df_osm = df_osm.to_crs(df_os.crs)
        print(f"OSM 数据已重新投影到 CRS: {df_osm.crs}")
    else:
        print(f"两个数据集的 CRS 匹配: {df_osm.crs}")


    # --- 为 OSM 数据创建 'geometry_osm' 副本 ---
    print("\n--- 准备 OSM 几何列副本 ---")
    if df_osm.geometry.name != 'geometry':
        df_osm = df_osm.set_geometry(df_osm.geometry.name)
    df_osm['geometry_osm'] = df_osm.geometry.copy() # 使用 .copy() 避免 SettingWithCopyWarning
    print("已为 df_osm 创建 'geometry_osm' 列作为几何副本。")

    # --- 3. 空间匹配 (基于相交) ---
    print("\n--- 执行空间匹配 ---")
    # 确保 rsuffix='osm' 将 OSM 的几何列重命名为 geometry_osm
    matched_buildings = gpd.sjoin(df_os, df_osm, how="inner", predicate="intersects", lsuffix='os', rsuffix='osm')
    print(f"初步匹配的建筑物对数量: {len(matched_buildings)}")

    # --- 空间匹配结果的额外检查点 ---
    print("\n--- 空间匹配结果详细检查 ---")
    print("matched_buildings 类型:", type(matched_buildings))
    print("matched_buildings 是否为 GeoDataFrame:", isinstance(matched_buildings, gpd.GeoDataFrame))
    if not matched_buildings.empty:
        print(f"matched_buildings 包含 {len(matched_buildings)} 个要素。")
        print("matched_buildings 列名:", matched_buildings.columns.tolist())
        print("matched_buildings 的几何列名 (OS源):", matched_buildings.geometry.name)
        if 'geometry_osm' in matched_buildings.columns:
            print("matched_buildings 中是否存在 'geometry_osm' 列: True")
            print("matched_buildings 'geometry_osm' 列数据类型:", matched_buildings['geometry_osm'].dtype)
            print("matched_buildings 'geometry_osm' 列前5行:")
            print(matched_buildings['geometry_osm'].head())
        else:
            print("重大错误：'geometry_osm' 列在匹配结果中不存在！IoU 计算将失败。")
            raise ValueError("'geometry_osm' 列在匹配结果中不存在。请检查数据和sjoin参数。")
    else:
        print("警告：matched_buildings 是空的，可能没有找到任何匹配项。无法进行 IoU 计算。")
        raise ValueError("没有找到相交的建筑物，无法进行 IoU 计算。请检查数据范围和CRS。")


    # --- 4. IoU (Intersection over Union) 计算 ---
    print("\n--- 计算 IoU 和面积 ---")
    print("正在尝试修复几何体...")
    matched_buildings.loc[:, 'geometry'] = matched_buildings['geometry'].buffer(0) # 使用 .loc 避免 SettingWithCopyWarning
    matched_buildings.loc[:, 'geometry_osm'] = matched_buildings['geometry_osm'].buffer(0) # 使用 .loc 避免 SettingWithCopyWarning
    print("几何体修复完成。")


    # 计算交集面积
    print("正在计算交集面积...")
    matched_buildings.loc[:, 'intersection_area'] = matched_buildings.apply( # 修正：lambda 表达式作为 apply 参数
        lambda row: row['geometry'].intersection(row['geometry_osm']).area,
        axis=1
    )
    print("交集面积计算完成。")

    # 计算联合面积
    print("正在计算联合面积...")
    matched_buildings.loc[:, 'union_area'] = matched_buildings.apply( # 修正：lambda 表达式作为 apply 参数
        lambda row: row['geometry'].union(row['geometry_osm']).area,
        axis=1
    )
    print("联合面积计算完成。")

    # 计算 IoU
    print("正在计算 IoU...")
    matched_buildings.loc[:, 'iou'] = matched_buildings.apply( # 修正：lambda 表达式作为 apply 参数
        lambda row: row['intersection_area'] / row['union_area'] if row['union_area'] > 0 else 0,
        axis=1
    )
    print("IoU 计算完成。")

    # 计算原始建筑物面积（在统一CRS后）
    matched_buildings.loc[:, 'area_os'] = matched_buildings['geometry'].area # 使用 .loc 避免 SettingWithCopyWarning
    matched_buildings.loc[:, 'area_osm'] = matched_buildings['geometry_osm'].area # 使用 .loc 避免 SettingWithCopyWarning

    print("IoU 和面积计算完成。")
    print("IoU 结果预览 (前5行):")
    print(matched_buildings[['area_os', 'area_osm', 'intersection_area', 'union_area', 'iou']].head())


    # --- 5. 筛选高 IoU 匹配 (可选) ---
    iou_threshold = 0.5
    high_iou_matches = matched_buildings[matched_buildings['iou'] >= iou_threshold].copy() # 使用 .copy() 避免 SettingWithCopyWarning
    print(f"\nIoU >= {iou_threshold} 的匹配数量: {len(high_iou_matches)}")
    print("高 IoU 匹配结果预览 (前5行):")
    print(high_iou_matches[['area_os', 'area_osm', 'iou']].head())


    # --- 6. 找出未匹配的建筑物 (可选) ---
    print("\n--- 找出未匹配的建筑物 ---")
    # 找出 OS 数据中未找到匹配的建筑物
    unmatched_os = df_os[~df_os['OBJECTID'].isin(matched_buildings['OBJECTID'])].copy() # 使用 .copy() 避免 SettingWithCopyWarning
    print(f"未匹配的 OS 建筑物数量: {len(unmatched_os)}")

    # 找出 OSM 数据中未找到匹配的建筑物
    unmatched_osm = df_osm[~df_osm.index.isin(matched_buildings['index_osm'])].copy() # 使用 .copy() 避免 SettingWithCopyWarning
    print(f"未匹配的 OSM 建筑物数量: {len(unmatched_osm)}")


    # --- 7. 保存结果 ---
    print("\n--- 保存结果 ---")
    output_matched_path = "/content/CB_matched_buildings_with_iou_and_area.gpkg"
    output_unmatched_os_path = "/content/CB__unmatched_os_buildings.gpkg"
    output_unmatched_osm_path = "/content/CB__unmatched_osm_buildings.gpkg"

    # 准备最终的匹配结果 GeoDataFrame
    final_matched_gdf = matched_buildings.copy() # 确保是副本

    # 将 'geometry_osm' 转换为 WKT 字符串
    if 'geometry_osm' in final_matched_gdf.columns:
        final_matched_gdf.loc[:, 'geometry_osm_wkt'] = final_matched_gdf['geometry_osm'].to_wkt()
        # 移除原始的 geometry_osm 列和辅助的 index_osm 列
        final_matched_gdf = final_matched_gdf.drop(columns=['index_osm', 'geometry_osm'])
    else:
        # 如果 geometry_osm 不存在，则只删除 index_osm
        final_matched_gdf = final_matched_gdf.drop(columns=['index_osm'])


    # --- 解决 fid 冲突和类型转换 for final_matched_gdf ---
    if 'fid' in final_matched_gdf.columns:
        final_matched_gdf.rename(columns={'fid': 'original_os_fid'}, inplace=True)
        print("已将 matched_buildings 中的 'fid' 列重命名为 'original_os_fid'。")
        if pd.api.types.is_float_dtype(final_matched_gdf['original_os_fid']):
            print("正在将 'original_os_fid' 列转换为整数类型...")
            final_matched_gdf.loc[:, 'original_os_fid'] = final_matched_gdf['original_os_fid'].fillna(-1).astype(int)
            final_matched_gdf.loc[:, 'original_os_fid'] = final_matched_gdf['original_os_fid'].replace(-1, pd.NA)
            print("'original_os_fid' 列转换完成。")

    # --- 解决 fid 冲突和类型转换 for unmatched_os ---
    if 'fid' in unmatched_os.columns:
        unmatched_os.rename(columns={'fid': 'original_os_fid'}, inplace=True)
        print("已将 unmatched_os 中的 'fid' 列重命名为 'original_os_fid'。")
        if pd.api.types.is_float_dtype(unmatched_os['original_os_fid']):
            print("正在将 unmatched_os 中的 'original_os_fid' 列转换为整数类型...")
            unmatched_os.loc[:, 'original_os_fid'] = unmatched_os['original_os_fid'].fillna(-1).astype(int)
            unmatched_os.loc[:, 'original_os_fid'] = unmatched_os['original_os_fid'].replace(-1, pd.NA)
            print("'original_os_fid' 列转换完成 (unmatched_os)。")

    # --- 解决 unmatched_osm 的多几何列问题 ---
    # unmatched_osm 初始包含 'geometry' 和 'geometry_osm'
    unmatched_osm.set_geometry('geometry', inplace=True) # 确保 'geometry' 是活动几何列

    if 'geometry_osm' in unmatched_osm.columns:
        print("正在将 unmatched_osm 中的 'geometry_osm' 列转换为 WKT 字符串...")
        unmatched_osm.loc[:, 'geometry_osm_wkt'] = unmatched_osm['geometry_osm'].to_wkt()
        unmatched_osm = unmatched_osm.drop(columns=['geometry_osm'])
        print("'geometry_osm' 列转换完成 (unmatched_osm)。")

    # 确保几何体列名为 'geometry' 且是活跃几何列 (对所有三个 GeoDataFrame)
    # 这一步是重复的，但可以作为最终检查，确保每个GeoDataFrame的活跃几何列设置正确
    final_matched_gdf.set_geometry('geometry', inplace=True)
    unmatched_os.set_geometry('geometry', inplace=True)
    unmatched_osm.set_geometry('geometry', inplace=True)

    # 保存文件
    final_matched_gdf.to_file(output_matched_path, driver="GPKG")
    print(f"匹配结果已保存到：{output_matched_path}")

    unmatched_os.to_file(output_unmatched_os_path, driver="GPKG")
    print(f"未匹配的 OS 建筑物已保存到：{output_unmatched_os_path}")

    unmatched_osm.to_file(output_unmatched_osm_path, driver="GPKG")
    print(f"未匹配的 OSM 建筑物已保存到：{output_unmatched_osm_path}")

    print("\n所有任务完成！您可以在左侧文件浏览器中找到输出文件，然后下载它们。")

except Exception as e:
    print(f"\n代码执行过程中发生错误：{e}")
    print("请检查错误信息，确保所有文件路径正确，数据格式匹配，且所有依赖库都已成功安装。")
    print("如果您反复遇到问题，请提供完整的错误信息 (Traceback)。")

正在尝试读取数据...
数据读取成功。

--- 数据加载检查 ---
成功读取 OSM 数据：包含 228661 个要素。
OSM 数据集预览 (前5行):
    osm_id  code    fclass                  name        type  \
0  3752659  1500  building  Abbey Leisure Centre        None   
1  3753306  1500  building                 Tesco        None   
2  3875404  1500  building                 Tesco      retail   
3  3875475  1500  building                  None  commercial   
4  3875491  1500  building               Harveys      retail   

                                            geometry  
0  POLYGON ((0.15405 52.20938, 0.1542 52.20946, 0...  
1  POLYGON ((0.18729 52.18697, 0.18729 52.18699, ...  
2  POLYGON ((0.14257 52.21122, 0.14355 52.21204, ...  
3  POLYGON ((0.14647 52.21186, 0.14687 52.21214, ...  
4  POLYGON ((0.14616 52.20933, 0.14626 52.2095, 0...  
OSM 数据集 CRS: EPSG:4326
OSM 数据集几何类型: ['Polygon' 'MultiPolygon']
成功读取 OS 数据：包含 85334 个要素。
OS 数据集预览 (前5行):
   OBJECTID            os_topo_to  os_topo_ve               bha_proces  \
0      36.0  osgb1000002500

In [7]:
# 单元格 4 (可选): 下载结果文件
from google.colab import files
files.download('/content/CB_matched_buildings_with_iou_and_area.gpkg')
files.download('/content/CB__unmatched_os_buildings.gpkg')
files.download('/content/CB__unmatched_osm_buildings.gpkg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 1. 将所有需要的文件打包成一个名为 BHresults.zip 的压缩文件
!zip /content/BHresults.zip /content/BH_matched_buildings_with_iou.gpkg /content/BH_unmatched_os.gpkg /content/BH_unmatched_osm.gpkg

# 2. 从 google.colab 导入 files 模块
from google.colab import files

# 3. 下载名为 BHresults.zip 的压缩包
files.download('/content/BHresults.zip')

	zip warning: name not matched: /content/BH_matched_buildings_with_iou.gpkg
	zip warning: name not matched: /content/BH_unmatched_os.gpkg
	zip warning: name not matched: /content/BH_unmatched_osm.gpkg

zip error: Nothing to do! (/content/BHresults.zip)


FileNotFoundError: Cannot find file: /content/BHresults.zip